In [1]:
import pandas as pd
import glob

### Import and concatenate all SO2 csv files into one

In [2]:
# Path to the directory containing CSV files 
csv_files_path = r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Data\SO2\*.csv" # Adjust this path as necessary 

# Use glob to get all CSV file paths 
csv_files = glob.glob(csv_files_path) 

# Use list comprehension to read all CSV files and concatenate them into a single DataFrame 
SO2_df = pd.concat([pd.read_csv(file,low_memory=False) for file in csv_files], ignore_index=True) 

# Now PM2_5_df contains all the data from the 20 CSV files 


In [3]:
# Drop unwanted columns
SO2_df.drop(['Parameter Code', 
            'Sample Duration', 
            'Pollutant Standard', 
            'Event Type', 
            'Observation Count', 
            'Observation Percent',
            'Method Code', 
            'Method Name',  
            'Local Site Name', 
            'Address', 
            'Date of Last Change'],axis=1,inplace=True)

In [4]:
# Drop Canada
SO2_df.drop(SO2_df[(SO2_df['State Code']=='CC')].index,inplace=True)

# Change State Code dtype to int64
SO2_df['State Code'] = SO2_df['State Code'].astype('int64')

# Drop Mexico, Virgin Islands and Puerto Rico
SO2_df.drop(SO2_df[(SO2_df['State Code']==80)|(SO2_df['State Code']==78)|(SO2_df['State Code']==72)].index,inplace=True)

In [5]:
# Aggregating so we have 1 measurement per station per day per parameter
SO2_df = SO2_df.groupby(['State Code', 
                       'State Name', 
                       'County Code', 
                       'County Name', 
                       'City Name', 
                       'Site Num', 
                       'Datum', 
                       'Latitude', 
                       'Longitude', 
                       'Date Local', 
                       'Parameter Name', 
                       'Units of Measure']).agg({'Arithmetic Mean':'mean', '1st Max Value':'max','1st Max Hour':'max','CBSA Name':'first','AQI':'mean'}).reset_index()

In [6]:
# Create unique identifier for each measurement
SO2_df['measurement_id'] = SO2_df['State Code'].apply(str) + '_' + SO2_df['County Code'].apply(str) + '_' + SO2_df['Site Num'].apply(str) + '_' + SO2_df['Date Local'].str.replace('-','')

In [7]:
# checking for the parameters included in the dataframe
SO2_df['Parameter Name'].unique()

array(['Sulfur dioxide'], dtype=object)

In [8]:
# Renaming column to match the value measured and droping unnecessary columns
SO2_df.rename(columns={'Arithmetic Mean':'SO2(ppb)','1st Max Value':'SO2(ppb) Max Value', '1st Max Hour':'SO2(ppb) Max Hour'},inplace=True)
SO2_df.drop(['Parameter Name', 'Units of Measure'],axis=1,inplace=True)

In [9]:
# Convert date column to datetime
SO2_df['Date Local'] = pd.to_datetime(SO2_df['Date Local'])

In [10]:
SO2_df.head()

,State Code,State Name,County Code,County Name,City Name,Site Num,Datum,Latitude,Longitude,Date Local,SO2(ppb),SO2(ppb) Max Value,SO2(ppb) Max Hour,CBSA Name,AQI,measurement_id
0,1,Alabama,33,Colbert,Not in a city,44,WGS84,34.690647,-87.821422,2004-01-01,3.112500,9.0,14,"Florence-Muscle Shoals, AL",13.0,1_33_44_20040101
1,1,Alabama,33,Colbert,Not in a city,44,WGS84,34.690647,-87.821422,2004-01-02,1.000000,1.0,2,"Florence-Muscle Shoals, AL",1.0,1_33_44_20040102
2,1,Alabama,33,Colbert,Not in a city,44,WGS84,34.690647,-87.821422,2004-01-03,1.125000,4.0,8,"Florence-Muscle Shoals, AL",6.0,1_33_44_20040103
3,1,Alabama,33,Colbert,Not in a city,44,WGS84,34.690647,-87.821422,2004-01-04,1.000000,1.0,2,"Florence-Muscle Shoals, AL",1.0,1_33_44_20040104
4,1,Alabama,33,Colbert,Not in a city,44,WGS84,34.690647,-87.821422,2004-01-05,3.072916,25.0,14,"Florence-Muscle Shoals, AL",36.0,1_33_44_20040105


In [11]:
#Saving final df to csv
#SO2_df.to_csv(r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Cleaning\all_SO2_clean.csv",index=False)
SO2_df.to_csv(r"C:\Users\stlva\Google Drive\Big blue data academy\main_branch\concatenated data\all_SO2_clean.csv",index=False)